In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cProfile, pstats, sys
import cv2
import sys
import csv
import pickle
from modules.path_planning.astar import AStar
from modules.path_planning.RRA import RRASEARCHTREE
from modules.path_planning.FIXED_ARA import ARA
from modules.path_planning.DLITE import DLITESEARCH
from modules.path_planning.RRT_star import RRT_Star
import time
import random
import math
import os
import copy
from os.path import isfile,join
static_or_dynamic = "STATIC"
pathPlannerNames = None
if static_or_dynamic == "STATIC":
    pathPlannerNames = ['RRT*', 'astar', 'ARA']
else:
    pathPlannerNames = ['RRA*', 'D* Lite', 'ARA']
    
fieldnames = ['Image Name', 'Image Size', 'Start Position', 'Goal Position']
fieldnames.extend(pathPlannerNames)
csvFileName = 'hi.csv'
pathMetrics = []
thewriter = None

   # thewriter.writerow({'Image Name': 'sample_pred0', 'Start Position': (15, 15), 'Goal Position': (19,19) })

startX, startY, endX, endY = 0,0,0,0
selectStart = True
selectEnd = True
img = None
im_Size = 600
img1 = None
prediction_matrix = None
path = None
path_runner = None

img_copy = None
ground_truth_im_copy = None
filename = None


def runPathPlanning(dynamic_or_static):
    global startX, startY, endX, endY
    startPos = (startX,startY)
    endPos = (endX,endY)
    global path
    global path_runner
    global img, img1, img_copy, ground_truth_im_copy
    global prediction_matrix
    global filename
    global pathPlannerNames
    algoPlanner_information = []
    prediction_matrix_copy = prediction_matrix.copy()
    for algoName in pathPlannerNames:
        temp_file_name = r'C:\Users\charles\Downloads\Path-Planning-On-Aerial-Images-main-20220523T022800Z-001\Path-Planning-On-Aerial-Images-main\MassachusettsPaths'

        start = time.time()

        if algoName == "astar":
            path_runner = AStar(startPos,endPos,copy.deepcopy(img),"image", prediction_matrix)
            path = path_runner.run()

        elif algoName == "ARA":
            path_runner = ARA(startPos, endPos, img, prediction_matrix, 0.75, img1)
            path = path_runner.static_path_main()

        elif algoName == "RRT_Star" or algoName == "RRT*":
            path_runner = RRT_Star(startPos, endPos, 5, 200, 50, copy.deepcopy(img))
            path = path_runner.run()

        elif algoName == "RRA*":
            path_runner = RRASEARCHTREE(startPos, endPos, img, img1)
            path = path_runner.ComputePath()

        elif algoName == "D* LITE":
            path_runner = DLITESEARCH(startPos, endPos, img, img1, prediction_matrix)
            path = path_runner.DLITERUN()

        else:
            print("No valid algorithm found for the algorithm name given...exiting")
            exit()

        end = time.time()
        duration = end - start
        solution_quality = 0.0

        similarity_Score = GTsimilarity(path, img1)

        if path != None:
            map_path = []
            first_point = path[0]
            second_point = None
            for pos in path:
                map_path.append(pos)
                if pos != first_point:
                    second_point = pos
                    cv2.line(img1, first_point, second_point, color=(0, 255, 0), thickness=5)
                    first_point = pos
                if pos != path[0]:
                    solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))
        else: 
            solution_quality = np.inf
            
        csvCellHolder = None
        if dynamic_or_static == "STATIC":
            csvCellHolder = [duration, solution_quality / np.linalg.norm(np.array([startX, startY])-np.array([endX, endY])), similarity_Score]
        else:
            csvCellHolder = [duration, solution_quality / np.linalg.norm(np.array([startX, startY])-np.array([endX, endY])), path_runner.replans]

        img1 = cv2.circle(img1, (startX, startY), radius=5, color=(255, 0, 0), thickness=-1)
        img1 = cv2.circle(img1, (endX, endY), radius=5, color=(0, 0, 255), thickness=-1)

        
        if dynamic_or_static == "STATIC":
        
            if algoName == "RRT_Star":
                temp_file_name += r'\STATIC\RRT\generated_path_' + filename    
            elif algoName == "ARA":
                temp_file_name += r'\STATIC\ARA\generated_path_' + filename
            elif algoName == "astar":
                temp_file_name += r'\STATIC\ASTAR\generated_path_' + filename
                
        else: 
 
            if algoName == "D* Lite":
                temp_file_name += r'\DYNAMIC\DSTARLITE\generated_path_' + filename    
            elif algoName == "ARA":
                temp_file_name += r'\DYNAMIC\ARA\generated_path_' + filename
            elif algoName == "RRA*":
                temp_file_name += r'\DYNAMIC\RRA\generated_path_' + filename
            
            
            
        cv2.imwrite(temp_file_name+'.jpg', img1)
        cv2.imwrite(temp_file_name+'segmentated'+'.jpg', img)
        
        algoPlanner_information.append(csvCellHolder)

        img1 = ground_truth_im_copy.copy()
        img = img_copy.copy()
        prediction_matrix = prediction_matrix_copy.copy()
    return algoPlanner_information

def GTsimilarity(path, gt_img):
    if path == None:
        return 0.0
    ratio = 0
    for a in path:
        if np.array_equal(gt_img[a[1]][a[0]], [255, 255, 255]):
            ratio += 1

    return (ratio/len(path)) * 100

def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    global img 
    global img1
    global prediction_matrix
    global im_Size
    global filename
    global img_copy
    global ground_truth_im_copy, static_or_dynamic
    global thewriter
    global startX, startY, endX, endY


    with open(csvFileName, 'w+') as csvfile:
        thewriter = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter = ';')
        thewriter.writeheader()

        imgFolder = "C:/Users/charles/Downloads/sample_predictions3/predictions"
        groundTruthFolder = "C:/Users/charles/Downloads/sample_predictions3/ground_truth"
        predictionFolder = "C:/Users/charles/Downloads/sample_predictions3/prediction_matrix"
        
        appender = []
        with open("pointsForPaths.csv", 'r') as file:
            csvreader = csv.reader(file, delimiter=";")
            for row in csvreader:
                if len(row) == 0:
                    continue
                appender.append(row)
        
        for point_arrays in appender:
            filename = point_arrays[0]
            with open('C:/Users/charles/Downloads/sample_predictions3/prediction_matrix/'+filename[0:-3]+'pkl', 'rb') as f:
                prediction_matrix = pickle.load(f)
            img = cv2.resize(cv2.imread(join(imgFolder, filename)), (im_Size, im_Size))
            img1 = cv2.resize(cv2.imread(join(groundTruthFolder, filename)), (im_Size, im_Size))
            ground_truth_im_copy = img1.copy()
            img_copy = img.copy()
            (startX, startY) =  eval(point_arrays[1])
            (endX, endY) =  eval(point_arrays[2])
            predicion_matrix = cv2.resize(prediction_matrix, (im_Size, im_Size))
            return_array = runPathPlanning(static_or_dynamic)
            thewriter.writerow({})
            




if __name__ == "__main__":
    main()



Path not found.
Beginning ARA* search from (77, 83) to (227, 358)
ARA* initial path has an accuracy of:  4.3478260869565215 %
Path not found.
Beginning ARA* search from (250, 13) to (586, 158)
ARA* initial path has an accuracy of:  9.042553191489363 %
Path not found.
Beginning ARA* search from (140, 37) to (165, 400)
ARA* initial path has an accuracy of:  7.4074074074074066 %
Path not found.
Beginning ARA* search from (103, 78) to (348, 394)
ARA* initial path has an accuracy of:  1.8469656992084433 %
Path not found.
Beginning ARA* search from (229, 21) to (511, 483)
ARA* initial path has an accuracy of:  2.5735294117647056 %
Path not found.
Beginning ARA* search from (116, 142) to (496, 48)
ARA* initial path has an accuracy of:  3.2842582106455263 %
Path not found.
Beginning ARA* search from (56, 349) to (508, 201)
ARA* initial path has an accuracy of:  4.635761589403973 %
Path not found.
Beginning ARA* search from (102, 356) to (520, 121)
ARA* initial path has an accuracy of:  4.28571


KeyboardInterrupt

